## Marketing A/B Testing

## What is A/B testing ?
 a/b testing is is one of the most used common methodsfor making data driven decisions for tech companyis .A/B tests compare two nearly identical version ofproduct or services to see which one performs better. in our case we will use the it to test our hypotheis

In [41]:
# module importation
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import math
import numpy
from statsmodels.stats.power import GofChisquarePower
from statsmodels.stats.contingency_tables import Table2x2

Marketing A/B testing dataset

Marketing companies want to run successful campaigns, but the market is complex and several options can work. So normally they run A/B tests, that is a randomized experimentation process wherein two or more versions of a variable (web page, page element, banner, etc.) are shown to different segments of people at the same time to determine which version leaves the maximum impact and drive business metrics.

###### The companies are interested in answering two questions:

Would the campaign be successful?

If the campaign was successful, how much of that success could be attributed to the ads?

In [40]:
# data importation 
df=pd.read_csv(r"C:\Users\Qwon\Desktop\Dev\linkedin_projects\AB Testing\dataset\marketing_AB.csv")
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


Data wrangling

In [42]:
# droping Unnamed: 0 columns
df=df.drop(columns="Unnamed: 0")

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 23.0+ MB


In [5]:
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


let understand our data

In [6]:
# number of columns
df.shape[0],df.tail()

(588101,
         user id test group  converted  total ads most ads day  most ads hour
 588096  1278437         ad      False          1      Tuesday             23
 588097  1327975         ad      False          1      Tuesday             23
 588098  1038442         ad      False          3      Tuesday             23
 588099  1496395         ad      False          1      Tuesday             23
 588100  1237779         ad      False          1      Tuesday             23)

In [7]:
px.bar(data_frame=df["most ads day"].value_counts(normalize=True))


**Amount of ads seen by person**

In [8]:


print(f'The minumum number of ads show is {df["total ads"].min()} and the maximum number of ads seen were {df["total ads"].max()} on mean number of ads were {int(df["total ads"].mean())}')

The minumum number of ads show is 1 and the maximum number of ads seen were 2065 on mean number of ads were 24


In [9]:
df["total ads"].min()

1

**test group**

In [10]:
df["test group"].value_counts()

test group
ad     564577
psa     23524
Name: count, dtype: int64

In [11]:
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


## Build Contingency Table

Now that the results are in a DataFrame, we can start pulling apart what we found. Let's start by making a table showing how many people converted   by the ads

In [18]:
df[["test group","converted"]].head()

,test group,converted
0,ad,False
1,ad,False
2,ad,False
3,ad,False
4,ad,False


In [35]:
data= pd.crosstab(index=df["test group"],
                  columns=df["converted"],
                  normalize=False)
data

converted,False,True
test group,,
ad,550154,14423
psa,23104,420


In [39]:
def build_contigency_bar():
    fig=px.bar(
        data_frame=data,
        barmode="group",
        title="converted by true or false"
        
    )
    return fig
build_contigency_bar()

In [62]:
#lets calculate the group size
chi_square_power=GofChisquarePower()
group_size=chi_square_power.solve_power(effect_size=.2,
                                        alpha=.05,
                                        power=.8
                                        
                                        )
# making the number to interger by truncating
group_size=math.ceil(group_size)
print(group_size)

197


In [44]:
data

converted,False,True
test group,,
ad,550154,14423
psa,23104,420


In [47]:
contingency_table=Table2x2(data.values)
contingency_table.table_orig

array([[550154,  14423],
       [ 23104,    420]], dtype=int64)

In [50]:
contingency_table.fittedvalues.round(0)


array([[550328.,  14249.],
       [ 22930.,    594.]])

Calculating the joint probabilities under independence for your `contingency_table`.

In [54]:
contingency_table.independence_probabilities.round(4)

array([[0.9358, 0.0242],
       [0.039 , 0.001 ]])

# chi-square test

In [61]:
chi_test=contingency_table.test_nominal_association()
print(chi_test)

df          1
pvalue      1.7053025658242404e-13
statistic   54.31805159138314


197


# Chi-squared Test:
Statistic: 54.32 indicates a strong association between the groups and whether they were shown ads.
P-value: 1.71×10−131.71×10−13 suggests that the association is statistically significant (typically, a p-value less than 0.05 is considered significant).
 ### we can agreer with  our altanative hypothesis that indeed show ads increases conversion